In [1]:
import itertools
import collections
import copy

    e---f---g---h---i---j---k (hallway)
         \ / \ / \ / \ / 
          a1  b1  c1  d1 (rooms)
          |   |   |   |
          a0  bO  cO  dO

Weights are 1 for e-f, j-k, a0-a1, b0-b1, c0-c1, and d0-d1, and 2 for all others.
The weights of 2 make it so we don't have to have rules about not stopping on a hallway 
space above one of the rooms.

In [2]:
graph = {'a0': {'a1': 1},
         'b0': {'b1': 1},
         'c0': {'c1': 1},
         'd0': {'d1': 1},
         'a1': {'a0': 1, 'f': 2, 'g': 2},
         'b1': {'b0': 1, 'g': 2, 'h': 2},
         'c1': {'c0': 1, 'h': 2, 'i': 2},
         'd1': {'d0': 1, 'i': 2, 'j': 2},
         'e': {'f': 1},
         'f': {'e': 1, 'a1': 2, 'g': 2},
         'g': {'f': 2, 'h': 2, 'a1': 2, 'b1': 2},
         'h': {'g': 2, 'i': 2, 'b1': 2, 'c1': 2},
         'i': {'h': 2, 'j': 2, 'c1': 2, 'd1': 2},
         'j': {'i': 2, 'k': 1, 'd1': 2},
         'k': {'j': 1},
        }
hallway = ['e', 'f', 'g', 'h', 'i', 'j', 'k']
rooms = ['a0', 'a1', 'b0', 'b1', 'c0', 'c1', 'd0', 'd1']
bottom_rooms = [room for room in rooms if '0' in room]
top_rooms = [room for room in rooms if '1' in room]

In [3]:
example_start = {'B0': 'a1', 'C1': 'b1', 'B1': 'c1', 'D1': 'd1',
                 'A0': 'a0', 'D0': 'b0', 'C0': 'c0', 'A1': 'd0'}

In [4]:
puzzle_start = {'D1': 'a1', 'B1': 'b1', 'C1': 'c1', 'A1': 'd1',
                'C0': 'a0', 'A0': 'b0', 'D0': 'c0', 'B0': 'd0'}

In [5]:
# from https://bradfieldcs.com/algos/graphs/dijkstras-algorithm/
import heapq


def calculate_distances(graph, starting_vertex):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[starting_vertex] = 0

    pq = [(0, starting_vertex)]
    while len(pq) > 0:
        current_distance, current_vertex = heapq.heappop(pq)

        # Nodes can get added to the priority queue multiple times. We only
        # process a vertex the first time we remove it from the priority queue.
        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # Only consider this new path if it's better than any path we've
            # already found.
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))

    return distances

## part 1 ##

In [7]:
move_costs = {'A0': 1, 'A1': 1, 'B0': 10, 'B1': 10,
              'C0': 100, 'C1': 100, 'D0': 1000, 'D1': 1000}

In [8]:
def is_done(positions):
    return all(k[0].lower() == v[0] for k,v in positions.items())

In [29]:
def not_in_place(positions):
    occupied = {v:k for k,v in positions.items()}
    in_place = []
    for rm in bottom_rooms: 
        if rm in occupied:
            if occupied[rm][0].lower() == rm[0]:
                in_place.append(occupied[rm])
                toprm = f'{rm[0]}1'
                if toprm in occupied:
                    if occupied[toprm][0].lower() == rm[0]:
                        in_place.append(occupied[toprm])
    return set(positions.keys()) - set(in_place)

In [12]:
def graph_with_positions(graph, positions):
    newgraph = copy.deepcopy(graph)
    occupied = positions.values()
    for vertex in graph:
        for endpt in graph[vertex]:
            if endpt in occupied:
                newgraph[vertex][endpt] = float('infinity')
    return newgraph

In [108]:
def allowed_movements(state, graph, max_cost=float('infinity')):
    positions, prev_cost = state
    if prev_cost > max_cost:
        return
    currgraph = graph_with_positions(graph, positions)
    can_move = not_in_place(positions)
    # If any amphipod can move to it's final spot, then that's the only move we want to make here
    for k in can_move:
        loc = positions[k]
        typ = k[0]
        if loc in hallway:
            # must move to the correct room, or stay in place
            dists = calculate_distances(currgraph, loc)
            # is the correct bottom room available?
            btm, top = f'{typ.lower()}0', f'{typ.lower()}1'
            if dists[btm] < float('infinity'):
                newpos = copy.deepcopy(positions)
                newpos[k] = btm
                cost = dists[btm]*move_costs[k] + prev_cost
                if cost < max_cost:
                    yield (newpos, cost)
                return
            # is the correct top room available?
            if dists[top] < float('infinity'):
                for kk, vv in positions.items():
                    if vv == btm:
                        btmtyp = kk[0]
                if btmtyp == typ:
                    # yes, it is
                    newpos = copy.deepcopy(positions)
                    newpos[k] = top
                    cost = dists[top]*move_costs[k] + prev_cost
                    if cost < max_cost:
                        yield (newpos, cost)
                    return
        else:
            # in a room, consider only direct moves to the correct room
            btm, top = f'{typ.lower()}0', f'{typ.lower()}1'
            dists = calculate_distances(currgraph, loc)
            # is the correct bottom room available?
            if dists[btm] < float('infinity'):
                newpos = copy.deepcopy(positions)
                newpos[k] = btm
                cost = dists[btm]*move_costs[k] + prev_cost
                if cost < max_cost:
                    yield (newpos, cost)
                return
            # is the correct top room available?
            if dists[top] < float('infinity'):
                for kk, vv in positions.items():
                    if vv == btm:
                        btmtyp = kk[0]
                if btmtyp == typ:
                    # yes, it is
                    newpos = copy.deepcopy(positions)
                    newpos[k] = top
                    cost = dists[top]*move_costs[k] + prev_cost
                    if cost < max_cost:
                        yield (newpos, cost)
                    return
    for k in can_move:
        loc = positions[k]
        if loc in rooms:
            dists = calculate_distances(currgraph, loc)
            # need to move into the hallway
            for i in hallway:
                if dists[i] < float('infinity'):
                    newpos = copy.deepcopy(positions)
                    newpos[k] = i
                    cost = dists[i]*move_costs[k] + prev_cost
                    if cost < max_cost:
                        yield (newpos, cost)


In [121]:
def solve1(positions, graph):
    states = [(positions, 0)]
    if is_done(positions):
        return 0
    final_costs = []
    i = 0
    while states:
        newstates = []
        for state in sorted(states, key=lambda s: s[1]):
            if final_costs:
                max_cost = min(final_costs)
            else:
                max_cost = float('infinity')
            for allowed in allowed_movements(state, graph, max_cost):
                pos, cost = allowed
                if is_done(pos):
                    final_costs.append(cost)
                else:
                    newstates.append(allowed)
        states = newstates
        i += 1
        print(i, len(states))
    return min(final_costs)

In [122]:
solve1(example_start, graph)

1 28
2 368
3 2234
4 7568
5 15595
6 21848
7 24237
8 27155
9 14388
10 2372
11 245
12 0


12521

In [115]:
solve1(puzzle_start, graph)

15338

## part 2 ##

    e---f---g---h---i---j---k (hallway)
         \ / \ / \ / \ / 
          a1  b1  c1  d1 (rooms)
          |   |   |   |
          a3  b3  c3  d3
          |   |   |   |
          a2  b2  c2  d2
          |   |   |   |
          a0  bO  cO  dO


In [125]:
graph2 = {'a0': {'a2': 1},
          'b0': {'b2': 1},
          'c0': {'c2': 1},
          'd0': {'d2': 1},
          'a2': {'a0': 1, 'a3': 1},
          'b2': {'b0': 1, 'b3': 1},
          'c2': {'c0': 1, 'c3': 1},
          'd2': {'d0': 1, 'd3': 1},
          'a3': {'a2': 1, 'a1': 1},
          'b3': {'b2': 1, 'b1': 1},
          'c3': {'c2': 1, 'c1': 1},
          'd3': {'d2': 1, 'd1': 1},
          'a1': {'a3': 1, 'f': 2, 'g': 2},
          'b1': {'b3': 1, 'g': 2, 'h': 2},
          'c1': {'c3': 1, 'h': 2, 'i': 2},
          'd1': {'d3': 1, 'i': 2, 'j': 2},
          'e': {'f': 1},
          'f': {'e': 1, 'a1': 2, 'g': 2},
          'g': {'f': 2, 'h': 2, 'a1': 2, 'b1': 2},
          'h': {'g': 2, 'i': 2, 'b1': 2, 'c1': 2},
          'i': {'h': 2, 'j': 2, 'c1': 2, 'd1': 2},
          'j': {'i': 2, 'k': 1, 'd1': 2},
          'k': {'j': 1},
        }
hallway = ['e', 'f', 'g', 'h', 'i', 'j', 'k']
rooms2 = [''.join(p) for p in itertools.product('abcd','0123')]
bottom_rooms = [room for room in rooms if '0' in room]

In [128]:
example2_start = {'B0': 'a1', 'C1': 'b1', 'B1': 'c1', 'D1': 'd1',
                  'D3': 'a3', 'C3': 'b3', 'B3': 'c3', 'A3': 'd3',
                  'D2': 'a2', 'B2': 'b2', 'A2': 'c2', 'C2': 'd2',
                  'A0': 'a0', 'D0': 'b0', 'C0': 'c0', 'A1': 'd0'}

In [132]:
puzzle2_start = {'D1': 'a1', 'B1': 'b1', 'C1': 'c1', 'A1': 'd1',
                'D3': 'a3', 'C3': 'b3', 'B3': 'c3', 'A3': 'd3',
                'D2': 'a2', 'B2': 'b2', 'A2': 'c2', 'C2': 'd2',
                'C0': 'a0', 'A0': 'b0', 'D0': 'c0', 'B0': 'd0'}

In [130]:
def not_in_place2(positions):
    occupied = {v:k for k,v in positions.items()}
    in_place = []
    goodcols = {'a', 'b', 'c', 'd'}
    for rmrow in (0, 2, 3, 1):
        for rm in [f'{col}{rmrow}' for col in 'abcd']:
            col = rm[0]
            if col not in goodcols:
                continue
            if rm not in occupied:
                goodcols.remove(col)
                continue
            if occupied[rm][0].lower() == rm[0]:
                in_place.append(occupied[rm])
            else:
                goodcols.remove(col)
    return set(positions.keys()) - set(in_place)

In [131]:
not_in_place2(example2_start)

{'A1',
 'A2',
 'A3',
 'B0',
 'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'D0',
 'D1',
 'D2',
 'D3'}

In [133]:
def available_homes(positions):
    ### FIXME ###
    available = []
    occupied = {v:k for k,v in positions.items()}
    for col in 'abcd':
        good = None
        for row in (0,2,3,1):
            rm = f'{col}{row}'
            if occupied[rm][0].lower() == col:
                good = row
        if good:
            pass

In [ ]:
def allowed_movements2(state, graph, max_cost=float('infinity')):
    positions, prev_cost = state
    if prev_cost > max_cost:
        return
    currgraph = graph_with_positions(graph, positions)
    can_move = not_in_place(positions)
    # If any amphipod can move to it's final spot, then that's the only move we want to make here
    for k in can_move:
        loc = positions[k]
        typ = k[0]
        if loc in hallway:
            # must move to the correct room, or stay in place
            dists = calculate_distances(currgraph, loc)
            # is the correct bottom room available?
            btm, top = f'{typ.lower()}0', f'{typ.lower()}1'
            if dists[btm] < float('infinity'):
                newpos = copy.deepcopy(positions)
                newpos[k] = btm
                cost = dists[btm]*move_costs[k] + prev_cost
                if cost < max_cost:
                    yield (newpos, cost)
                return
            # is the correct top room available?
            if dists[top] < float('infinity'):
                for kk, vv in positions.items():
                    if vv == btm:
                        btmtyp = kk[0]
                if btmtyp == typ:
                    # yes, it is
                    newpos = copy.deepcopy(positions)
                    newpos[k] = top
                    cost = dists[top]*move_costs[k] + prev_cost
                    if cost < max_cost:
                        yield (newpos, cost)
                    return
        else:
            # in a room, consider only direct moves to the correct room
            btm, top = f'{typ.lower()}0', f'{typ.lower()}1'
            dists = calculate_distances(currgraph, loc)
            # is the correct bottom room available?
            if dists[btm] < float('infinity'):
                newpos = copy.deepcopy(positions)
                newpos[k] = btm
                cost = dists[btm]*move_costs[k] + prev_cost
                if cost < max_cost:
                    yield (newpos, cost)
                return
            # is the correct top room available?
            if dists[top] < float('infinity'):
                for kk, vv in positions.items():
                    if vv == btm:
                        btmtyp = kk[0]
                if btmtyp == typ:
                    # yes, it is
                    newpos = copy.deepcopy(positions)
                    newpos[k] = top
                    cost = dists[top]*move_costs[k] + prev_cost
                    if cost < max_cost:
                        yield (newpos, cost)
                    return
    for k in can_move:
        loc = positions[k]
        if loc in rooms:
            dists = calculate_distances(currgraph, loc)
            # need to move into the hallway
            for i in hallway:
                if dists[i] < float('infinity'):
                    newpos = copy.deepcopy(positions)
                    newpos[k] = i
                    cost = dists[i]*move_costs[k] + prev_cost
                    if cost < max_cost:
                        yield (newpos, cost)